## VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import warnings
warnings.simplefilter('ignore')

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
source_file = './output_data/output_cities_AR.csv'
cities_df = pd.read_csv(source_file,sep = ",", low_memory=False)
cities_df['Date'] = pd.to_datetime(cities_df['Date'], unit='s')
cities_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


In [3]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City        0 non-null      object        
 1   Latitude    0 non-null      object        
 2   Longitude   0 non-null      object        
 3   Max Temp    0 non-null      object        
 4   Humidity    0 non-null      object        
 5   Cloudiness  0 non-null      object        
 6   Wind Speed  0 non-null      object        
 7   Country     0 non-null      object        
 8   Date        0 non-null      datetime64[ns]
dtypes: datetime64[ns](1), object(8)
memory usage: 0.0+ bytes


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df['Humidity']

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,
                                 weights = humidity, dissipating = False,
                                 max_intensity = 300, point_radius = 5)
fig.add_layer(heat_layer)
fig

TraitError: The 'locations' trait of a WeightedHeatmap instance must be of length 1 <= L <= 9223372036854775807, but a value of [] was specified.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
hotel_df = cities_df.loc[(cities_df['Max Temp'] < 80) & 
                         (cities_df['Max Temp'] > 70) 
                              & (cities_df['Wind Speed'] < 10) & (cities_df['Humidity'] <=50)].copy() # I like clouds

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# * Store into variable named `hotel_df`.
# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ""

hotel_df.reset_index(drop=True).head()

In [ ]:
# TARGET / COORDINATES
radius = 5000

# URL
base_url_nearby = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# PARAMETERS
params = {
    "types": "lodging",
    "radius": radius,
    "key": g_key}

for index,row in hotel_df.iterrows():
    lat=row["Latitude"]
    lng=row["Longitude"]
    params["location"]=f"{lat},{lng}"
    
# RUN REQUEST
    response = requests.get(base_url_nearby, params)
    
# # TEST RESULTS
#     print(response.url)
    results = response.json()
#    print(results["results"][0]["name"])
# print(results["results"][0]["vicinity"])

# # PRINT RESULTS

    try:
     # record the hotel name
        hotel_df['Hotel Name'][index] = results["results"][0]["name"]
    except IndexError:
        print(f"No hotels available near {hotel_df['City'][index]}.\n")
        hotel_df['Hotel Name'][index] = "none available"
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
hotel_info

In [ ]:
new_map = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(new_map)
fig